In [14]:
import pandas as pd
import numpy as np
from datetime import datetime

In [15]:
dataset = pd.read_excel(r"Danh sách cấp tỉnh kèm theo quận huyện, phường xã ___15_03_2023.xls", sheet_name="Sheet1", header=0)

WARNING *** file size (2453055) not 512 + multiple of sector size (512)


In [16]:
dataset.head()

,Tỉnh Thành Phố,Mã TP,Quận Huyện,Mã QH,Phường Xã,Mã PX,Cấp,Tên Tiếng Anh
0,Thành phố Hà Nội,1,Quận Ba Đình,1,Phường Phúc Xá,1.0,Phường,NaN
1,Thành phố Hà Nội,1,Quận Ba Đình,1,Phường Trúc Bạch,4.0,Phường,NaN
2,Thành phố Hà Nội,1,Quận Ba Đình,1,Phường Vĩnh Phúc,6.0,Phường,Vinh Phuc Commune
3,Thành phố Hà Nội,1,Quận Ba Đình,1,Phường Cống Vị,7.0,Phường,NaN
4,Thành phố Hà Nội,1,Quận Ba Đình,1,Phường Liễu Giai,8.0,Phường,Lieu Giai Commune


In [17]:
dataset.columns

Index(['Tỉnh Thành Phố', 'Mã TP', 'Quận Huyện', 'Mã QH', 'Phường Xã', 'Mã PX',
       'Cấp', 'Tên Tiếng Anh'],
      dtype='object')

In [18]:
dataset.drop(['Tên Tiếng Anh', "Cấp"], axis=1, inplace=True)

In [19]:
dataset.head()

,Tỉnh Thành Phố,Mã TP,Quận Huyện,Mã QH,Phường Xã,Mã PX
0,Thành phố Hà Nội,1,Quận Ba Đình,1,Phường Phúc Xá,1.0
1,Thành phố Hà Nội,1,Quận Ba Đình,1,Phường Trúc Bạch,4.0
2,Thành phố Hà Nội,1,Quận Ba Đình,1,Phường Vĩnh Phúc,6.0
3,Thành phố Hà Nội,1,Quận Ba Đình,1,Phường Cống Vị,7.0
4,Thành phố Hà Nội,1,Quận Ba Đình,1,Phường Liễu Giai,8.0


In [20]:
dataset['Mã TP'].unique()

array([ 1,  2,  4,  6,  8, 10, 11, 12, 14, 15, 17, 19, 20, 22, 24, 25, 26,
       27, 30, 31, 33, 34, 35, 36, 37, 38, 40, 42, 44, 45, 46, 48, 49, 51,
       52, 54, 56, 58, 60, 62, 64, 66, 67, 68, 70, 72, 74, 75, 77, 79, 80,
       82, 83, 84, 86, 87, 89, 91, 92, 93, 94, 95, 96])

In [21]:
dataset[dataset.isnull().any(axis=1)]

,Tỉnh Thành Phố,Mã TP,Quận Huyện,Mã QH,Phường Xã,Mã PX
3797,Thành phố Hải Phòng,31,Huyện Bạch Long Vĩ,318,NaN,NaN
6208,Tỉnh Quảng Trị,45,Huyện Cồn Cỏ,471,NaN,NaN
6406,Thành phố Đà Nẵng,48,Huyện Hoàng Sa,498,NaN,NaN
6821,Tỉnh Quảng Ngãi,51,Huyện Lý Sơn,536,NaN,NaN
8686,Tỉnh Bà Rịa - Vũng Tàu,77,Huyện Côn Đảo,755,NaN,NaN


In [22]:
PROVINCES = []
DISTRICTS = []
WARDS = []
index_p = 1
index_d = 1
index_w = 1

province_ids = dataset['Mã TP'].unique()
for province_id in province_ids:
    provinces =  dataset[dataset['Mã TP'] == province_id]

    # add
    item = provinces.iloc[0]
    p = [index_p, int(item["Mã TP"]), str(item["Tỉnh Thành Phố"])]
    PROVINCES.append(p)

    district_ids = provinces["Mã QH"].unique()
    for district_id in district_ids:
        districts =  provinces[provinces['Mã QH'] == district_id]

        # add
        item = districts.iloc[0]
        d = [index_d, int(item["Mã QH"]), str(item["Quận Huyện"]), index_p]
        DISTRICTS.append(d)

        for index_3, ward in districts.iterrows():
            if np.isnan(ward["Mã PX"]):
                continue
            w = [index_w, int(ward["Mã QH"]), str(ward["Phường Xã"]), index_d]
            WARDS.append(w)
            index_w += 1

        index_d += 1
    index_p += 1

In [ ]:
_p = pd.DataFrame(PROVINCES)
_p.columns = ["Id", "Code", "Name"]
_p.to_csv("provinces.csv", index=False, header=False)

In [ ]:
_d = pd.DataFrame(DISTRICTS)
_d.columns = ["Id", "Code", "Name", "ProvinceId"]
_d.to_csv("districts.csv", index=False, header=False)

In [ ]:
_w = pd.DataFrame(WARDS)
_w.columns = ["Id", "Code", "Name", "DistrictId"]
_w.to_csv("wards.csv", index=False, header=False)

In [ ]:
from datetime import datetime
# NEO
_p["Status"] = 1 
_p["IsDeleted"] = False
_p["CreatedAt"] = datetime.now().today()
_p["UpdatedAt"] = np.nan
_p.to_csv("neo-provinces.csv", index=False, header=True)

_d["Status"] = 1 
_d["IsDeleted"] = False
_d["CreatedAt"] = datetime.now().today()
_d["UpdatedAt"] = np.nan
_d.to_csv("neo-districts.csv", index=False, header=True)

_w["Status"] = 1 
_w["IsDeleted"] = False
_w["CreatedAt"] = datetime.now().today()
_w["UpdatedAt"] = np.nan
_w.to_csv("neo-wards.csv", index=False, header=True)